# Download Model and upload to Object Bucket

### Get Packages

In [1]:
!pip install --upgrade huggingface_hub -q
!pip install transformers -q

import torch
import os
import boto3


[notice] A new release of pip available: 22.2.2 -> 24.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.2.2 -> 24.0
[notice] To update, run: pip install --upgrade pip


### Login to Hugging Face

In [2]:
!huggingface-cli login --token $hf

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /opt/app-root/src/.cache/huggingface/token
Login successful


### Check GPU

In [4]:
print(torch.cuda.is_available())
print(torch.cuda.device_count())
#print(torch.cuda.get_device_name(0))

False
0


### Download Model from HuggingFace

In [11]:
save_dir = "/opt/app-root/src/mistral"

/opt/app-root/src/openshift-notebooks/notebooks


In [9]:
!huggingface-cli download --local-dir /opt/app-root/src/mistral mistralai/Mistral-7B-Instruct-v0.2 --quiet

Fetching 16 files: 100%|██████████████████████| 16/16 [00:00<00:00, 1288.35it/s]
/opt/app-root/src/mistral


### To check S3 values

In [62]:
# !echo $AWS_ACCESS_KEY_ID
# !echo $AWS_SECRET_ACCESS_KEY
# !echo $AWS_S3_BUCKET
# !echo $AWS_S3_ENDPOINT
# !echo $AWS_DEFAULT_REGION

## Set up minio CLI

In [12]:
!curl https://dl.min.io/client/mc/release/linux-amd64/mc --create-dirs -o ./minio-binaries/mc
!chmod +x ./minio-binaries/mc
!export PATH=$PATH:/opt/app-root/src/openshift-notebooks/notebooks/minio-binaries # Does not work from jupyter notebook

!./minio-binaries/mc alias set minio $AWS_S3_ENDPOINT --api "S3v4" $AWS_ACCESS_KEY_ID $AWS_SECRET_ACCESS_KEY
!./minio-binaries/mc alias ls minio

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 25.2M  100 25.2M    0     0  16.0M      0  0:00:01  0:00:01 --:--:-- 16.0M
]11;?\Added `minio` successfully.
]11;?\minio
  URL       : http://minio-service.minio.svc.cluster.local:9000
  AccessKey : minio
  SecretKey : minio123
  API       : S3v4
  Path      : auto



### Upload model to the bucket

In [25]:
!./minio-binaries/mc cp -r /opt/app-root/src/mod minio/mod/

...onfig.json: 27.47 GiB / 27.47 GiB ┃▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓┃ 103.57 MiB/s 4m31s

### Verify

In [ ]:
!./minio-binaries/mc cp -r /opt/app-root/src/mod minio/mod/

In [27]:
!./minio-binaries/mc ls minio/mod/mod/

]11;?\[2024-05-05 09:53:22 UTC]     0B mistralai/


In [28]:
!./minio-binaries/mc ls minio/mod/mod/mistralai

]11;?\[2024-05-05 09:48:17 UTC] 1.5KiB STANDARD .gitattributes
[2024-05-05 09:53:34 UTC]     0B .huggingface/
[2024-05-05 09:53:34 UTC]     0B Mistral-7B-Instruct-v0.2/


In [29]:
!./minio-binaries/mc ls minio/mod/mod/mistralai/Mistral-7B-Instruct-v0.2/

]11;?\[2024-05-05 09:48:17 UTC] 3.7KiB STANDARD README.md
[2024-05-05 09:48:17 UTC]   596B STANDARD config.json
[2024-05-05 09:48:17 UTC]   111B STANDARD generation_config.json
[2024-05-05 09:52:38 UTC] 4.6GiB STANDARD model-00001-of-00003.safetensors
[2024-05-05 09:52:39 UTC] 4.7GiB STANDARD model-00002-of-00003.safetensors
[2024-05-05 09:51:44 UTC] 4.2GiB STANDARD model-00003-of-00003.safetensors
[2024-05-05 09:48:17 UTC]  24KiB STANDARD model.safetensors.index.json
[2024-05-05 09:52:38 UTC] 4.6GiB STANDARD pytorch_model-00001-of-00003.bin
[2024-05-05 09:52:38 UTC] 4.7GiB STANDARD pytorch_model-00002-of-00003.bin
[2024-05-05 09:52:40 UTC] 4.7GiB STANDARD pytorch_model-00003-of-00003.bin
[2024-05-05 09:48:17 UTC]  23KiB STANDARD pytorch_model.bin.index.json
[2024-05-05 09:48:17 UTC]    72B STANDARD special_tokens_map.json
[2024-05-05 09:48:17 UTC] 1.7MiB STANDARD tokenizer.json
[2024-05-05 09:48:17 UTC] 482KiB STANDARD tokenizer.model
[2024-05-05 09:48:17 UTC] 1.4KiB STANDARD tokeni

In [21]:
!./minio-binaries/mc ls minio/$AWS_S3_BUCKET/mistral/

]11;?\[2024-05-05 07:39:03 UTC] 1.5KiB STANDARD .gitattributes
[2024-05-05 07:39:03 UTC] 3.7KiB STANDARD README.md
[2024-05-05 07:39:03 UTC]   596B STANDARD config.json
[2024-05-05 07:39:03 UTC]   111B STANDARD generation_config.json
[2024-05-05 07:42:47 UTC] 4.6GiB STANDARD model-00001-of-00003.safetensors
[2024-05-05 07:42:50 UTC] 4.7GiB STANDARD model-00002-of-00003.safetensors
[2024-05-05 07:42:31 UTC] 4.2GiB STANDARD model-00003-of-00003.safetensors
[2024-05-05 07:39:03 UTC]  24KiB STANDARD model.safetensors.index.json
[2024-05-05 07:42:50 UTC] 4.6GiB STANDARD pytorch_model-00001-of-00003.bin
[2024-05-05 07:42:50 UTC] 4.7GiB STANDARD pytorch_model-00002-of-00003.bin
[2024-05-05 07:42:51 UTC] 4.7GiB STANDARD pytorch_model-00003-of-00003.bin
[2024-05-05 07:39:03 UTC]  23KiB STANDARD pytorch_model.bin.index.json
[2024-05-05 07:39:03 UTC]    72B STANDARD special_tokens_map.json
[2024-05-05 07:39:03 UTC] 1.7MiB STANDARD tokenizer.json
[2024-05-05 07:39:03 UTC] 482KiB STANDARD tokeniz

# ROUGH

In [ ]:
mistralai/Mistral-7B-Instruct-v0.2

In [23]:
!./minio-binaries/mc buckets -h

]11;?\mc: <ERROR> `buckets` is not a recognized command. Get help using `--help` flag. 


In [32]:
!./minio-binaries/mc rm minio/models

]11;?\

In [33]:
!./minio-binaries/mc rb minio/models --force --dangerous

]11;?\Removed `minio/models` successfully.
